In [2]:
import ETL.utils as utils
import pandas as pd
logger = utils.setup_logging()


In [4]:
import ETL.ETL_dim_card as ETL_dim_card
import ETL.config as config
import importlib
importlib.reload(config)
importlib.reload(extract)

cards= extract.extract_cards_data(config.CARDS_FILE_LOCAL)
# users = extract.extract_users_data()
# transactions = extract.extract_transactions_data()



TypeError: extract_cards_data() takes 0 positional arguments but 1 was given

In [4]:
def extract_cards_data() -> pd.DataFrame:
    """
    This function extracts card data from a CSV file and returns it as a DataFrame.
    It logs the number of records loaded or any errors encountered during the process.
    The function uses the path defined in the config module to locate the CSV file.

    Args:
        None
    Returns:
        pd.DataFrame: DataFrame containing card data if successful, None otherwise.
    """
    try:
        df = pd.read_csv(r'C:\Users\jimmy\Documents\Data Projects\Project\Financial Transactions Project\data\cards_data.csv')
        logger.info(f"✅ Extracted cards_data.csv with {len(df)} records")
        return df
    except Exception as e:
        logger.error(f"❌ Error extracting cards_data.csv: {e}")
        raise 


In [10]:
df1 = extract_cards_data()
df1['has_chip'] = df1['has_chip'].replace({'YES': 'Yes', 'NO': 'No'})
df1

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,Yes,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,Yes,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,Yes,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,No,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,Yes,1,$28,09/2008,2009,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,5361,185,Amex,Credit,300609782832003,01/2024,663,Yes,1,$6900,11/2000,2013,No
6142,2711,185,Visa,Credit,4718517475996018,01/2021,492,Yes,2,$5700,04/2012,2012,No
6143,1305,1007,Mastercard,Credit,5929512204765914,08/2020,237,No,2,$9200,02/2012,2012,No
6144,743,1110,Mastercard,Debit,5589768928167462,01/2020,630,Yes,1,$28074,01/2020,2020,No


In [26]:
df1.sample(10)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
3592,251,649,Visa,Debit,4827129780511983,03/2024,941,Yes,1,$8280,01/2010,2010,No
3593,2105,649,Discover,Credit,6267383831462523,12/2012,640,Yes,1,$0,03/2006,2008,No
4299,1324,58,Mastercard,Debit,5850540702416613,05/2022,840,Yes,2,$23838,02/2013,2013,No
3559,3601,1125,Visa,Debit,4218957266147893,05/2021,909,Yes,2,$29200,06/2015,2015,No
1361,4560,236,Visa,Debit,4003314505746426,07/2021,531,Yes,2,$6076,09/2004,2011,No
4200,4367,216,Mastercard,Debit (Prepaid),5094200042825369,10/2023,696,No,1,$80,08/2011,2011,No
11,2029,1718,Mastercard,Debit,5804499644308599,07/2023,258,No,2,$27480,03/2002,2008,No
2412,4588,509,Visa,Debit,4262181069766792,07/2022,519,Yes,1,$12721,09/2005,2015,No
1609,1351,428,Amex,Credit,340213655737651,11/2024,108,Yes,2,$8800,02/2015,2015,No
5137,147,256,Visa,Credit,4577476596554924,12/2010,553,Yes,2,$14300,01/2007,2007,No


In [ ]:
def cards_transform(df: pd.DataFrame) -> pd.DataFrame:
    """
    Tranform the DataFrame.
    Args:
        df (pd.DataFrame): The DataFrame to transform.
    Returns:
        pd.DataFrame: The DataFrame with specified transformations.
    """
    try:
        columns_to_remove = ['client_id', 'card_number', 'cvv']
        columns_to_rename = {'id': 'card_id','acct_open_date': 'account_open_date'}
        df_cleaned = df.copy()
        df_cleaned = df.drop(columns=columns_to_remove)
        df_cleaned = df_cleaned.rename(columns=columns_to_rename)
        df_cleaned['credit_limit'] = df_cleaned['credit_limit'].str.extract(r'(\d+)').astype(int)
        df_cleaned['has_chip'] = df_cleaned['has_chip'].replace({'YES': 'Yes', 'NO': 'No'})
        df_cleaned['card_id'] = df_cleaned['card_id'].astype(str)
        df_cleaned['expires'] = pd.to_datetime(df_cleaned['expires'], errors='raise',format= '%m/%Y').dt.strftime('%Y%m')
        df_cleaned['account_open_date'] = pd.to_datetime(df_cleaned['account_open_date'], errors='raise',format= '%m/%Y').dt.strftime('%Y%m')

        logger.info(f"Removed columns {columns_to_remove} from CARDS data.")
        return df_cleaned
    
    except Exception as e:
        logger.error(f"An error occurred while transforming CARDS data: {e}")
        raise
df2 = cards_transform(df1)

In [25]:
dupli = df2.loc[df2.duplicated(subset=['expires','has_chip'] , keep=False),['card_id','expires','has_chip']]
dupli

,card_id,expires,has_chip
0,4524,202212,Yes
1,2731,202012,Yes
2,3701,202402,Yes
3,42,202408,No
4,4659,200903,Yes
...,...,...,...
6141,5361,202401,Yes
6142,2711,202101,Yes
6143,1305,202008,No
6144,743,202001,Yes


In [14]:
df2.sample(10)

,card_id,card_brand,card_type,expires,has_chip,num_cards_issued,credit_limit,account_open_date,year_pin_last_changed,card_on_dark_web,id,acct_open_date
2059,2857,Mastercard,Debit,201002,Yes,1,4067,05/2004,2020,No,2857,200405
417,108,Visa,Credit,202305,Yes,1,9800,01/2006,2015,No,108,200601
2713,2458,Mastercard,Debit,202407,Yes,2,2054,04/2004,2015,No,2458,200404
3799,2053,Visa,Debit,202405,Yes,2,17405,03/2003,2003,No,2053,200303
2836,2484,Mastercard,Credit,202001,Yes,2,8100,04/2005,2008,No,2484,200504
5970,5447,Visa,Debit,200708,Yes,2,8024,11/2005,2011,No,5447,200511
3951,3450,Visa,Credit,202206,Yes,2,25400,06/2009,2012,No,3450,200906
1969,1851,Visa,Debit,202309,Yes,2,16998,02/2020,2020,No,1851,202002
2466,3681,Visa,Debit,202211,Yes,2,9952,07/2002,2010,No,3681,200207
3220,4722,Mastercard,Debit (Prepaid),202010,Yes,1,77,09/2009,2009,No,4722,200909


In [ ]:
#test1['expires']= pd.to_datetime(test1['expires'])
test1.sample(5)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
840,3983,586,Visa,Credit,4095430974536550,03/2021,614,YES,2,$20300,07/2011,2011,No
1991,2964,179,Mastercard,Debit,5078628761076723,11/2024,417,YES,2,$17417,05/2008,2009,No
1803,4071,565,Visa,Debit,4125650435743087,09/2024,385,NO,1,$16080,07/2018,2018,No
4072,3110,913,Visa,Credit,4526538061468985,03/2020,180,YES,1,$6900,05/2011,2011,No
262,10,1783,Mastercard,Debit,5513208341097350,12/2023,789,YES,1,$16597,01/1999,2009,No


In [ ]:
# count missing values
cards.isnull().sum()

id                       0
client_id                0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit             0
acct_open_date           0
year_pin_last_changed    0
card_on_dark_web         0
dtype: int64

In [ ]:
#check for duplicates
import pandas as pd
def check_duplicates(df: pd.DataFrame)-> None:
    """
    Check for duplicate rows in the DataFrame based on specified columns.
    Args:
        df (pd.DataFrame): The DataFrame to check for duplicates.
    Returns:
        None
    """
    try:
        
        critical_columns = ['id', 'client_id']
        duplicates = df.duplicated(subset=critical_columns).sum()
        
        if duplicates > 0:
            logger.info(f"There are {duplicates} duplicate records in the data based on columns {critical_columns}.")
        else:
            logger.info(f"No duplicate records found in the data based on columns {critical_columns}.")

    except Exception as e:
        logger.error(f"An error occurred while checking for duplicates: {e}")
        raise



In [ ]:
def cards_check_missing_values(df: pd.DataFrame) -> None:
    """
    Check for missing values in the DataFrame and log the results.
    Args:
        df (pd.DataFrame): The DataFrame to check for missing values.
    Returns:
        None
    """
    try:
        missing_values = df.isnull().sum()
        missing_columns = missing_values[missing_values > 0]
        
        if not missing_columns.empty:
            for column, count in missing_columns.items():
                logger.info(f"Column '{column}' has {count} missing values in CARDS data.")
            
            if all(col in missing_columns for col in ['id', 'client_id']):
                logger.error("Critical columns 'id' and 'client_id' in CARDS data contain missing values, which may affect data integrity.")
        else:
            logger.info("No missing values found in CARDS data.")
    
    except Exception as e:
        logger.error(f"An error occurred while checking for missing values in CARDS data: {e}")
        raise


In [ ]:
cards.sample(5)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
1324,3721,1211,Visa,Debit,4904264423390551,02/2008,240,YES,2,$119,07/2004,2012,No
4182,2431,1820,Mastercard,Credit,5023190732647203,01/2022,785,YES,1,$7900,04/2002,2013,No
1933,3996,1587,Mastercard,Debit,5638793020721462,02/2020,531,YES,2,$35968,07/2012,2012,No
4842,4472,1759,Mastercard,Debit,5582577330932323,01/2024,605,YES,2,$14801,08/2018,2018,No
68,748,777,Visa,Debit,4832328468851061,08/2023,580,YES,1,$68400,01/2020,2020,No


In [ ]:
def cards_remove_rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove and rename specified columns from the DataFrame.
    Args:
        df (pd.DataFrame): The DataFrame from which to remove columns.
    Returns:
        pd.DataFrame: The DataFrame with specified columns removed.
    """
    try:
        columns_to_remove = ['client_id', 'card_number', 'cvv']
        columns_to_rename = {'id': 'card_id','acct_open_date': 'account_open_date'}
        df_cleaned = df.copy()
        df_cleaned = df.drop(columns=columns_to_remove)
        df_cleaned = df_cleaned.rename(columns=columns_to_rename)
        logger.info(f"Removed columns {columns_to_remove} from CARDS data.")
        return df_cleaned
    
    except Exception as e:
        logger.error(f"An error occurred while removing columns from CARDS data: {e}")
        raise

In [ ]:
cards =  cards_remove_rename_columns(cards)
cards.sample(5)


,card_id,card_brand,card_type,expires,has_chip,num_cards_issued,credit_limit,account_open_date,year_pin_last_changed,card_on_dark_web
5066,307,Visa,Credit,06/2017,YES,2,$2800,01/2012,2012,No
5544,2037,Mastercard,Debit,11/2020,YES,1,$4701,03/2002,2017,No
3010,1629,Mastercard,Debit,01/2024,YES,2,$7458,02/2020,2020,No
3477,3203,Mastercard,Debit,02/2020,NO,1,$8993,05/2019,2019,No
5398,1153,Mastercard,Debit (Prepaid),07/2022,YES,2,$20,02/2008,2017,No


In [ ]:
cards.loc[:, 'acct_open_date'].str[3:].value_counts().sort_index(ascending=True)

acct_open_date
1991       3
1992       2
1993       3
1994      10
1995       6
1996      16
1997      27
1998      42
1999      61
2000      64
2001     121
2002     147
2003     197
2004     246
2005     300
2006     359
2007     408
2008     487
2009     466
2010     545
2011     335
2012     188
2013     199
2014     178
2015     148
2016     120
2017     101
2018     101
2019      88
2020    1178
Name: count, dtype: int64

In [ ]:
cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   card_id                6146 non-null   int64 
 1   card_brand             6146 non-null   object
 2   card_type              6146 non-null   object
 3   expires                6146 non-null   object
 4   has_chip               6146 non-null   object
 5   num_cards_issued       6146 non-null   int64 
 6   credit_limit           6146 non-null   object
 7   account_open_date      6146 non-null   object
 8   year_pin_last_changed  6146 non-null   int64 
 9   card_on_dark_web       6146 non-null   object
dtypes: int64(3), object(7)
memory usage: 480.3+ KB


In [ ]:
# here, I need to set up datatype conversion for the columns

def cards_convert_datatypes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert the data types of specified columns in the DataFrame.
    Args:
        df (pd.DataFrame): The DataFrame to convert data types.
    Returns:
        pd.DataFrame: The DataFrame with converted data types.
    """
    try:
        cards['credit_limit'] = cards['credit_limit'].str.extract(r'(\d+)').astype(int)
        cards['expires'] = pd.to_datetime(cards['expires'], errors='raise',format= '%m/%Y').dt.strftime('%Y%m')
        cards['account_open_date'] = pd.to_datetime(cards['account_open_date'], errors='raise',format= '%m/%Y').dt.strftime('%Y%m')
        logger.info('transform datatypes in specific colums ok')
    except Exception as e:
        logger.error(f"An error occurred while converting data types in CARDS data: {e}")
        raise

In [ ]:
cards_convert_datatypes(cards)

In [ ]:
import pandas as pd
import logging
from sqlalchemy.engine import Engine
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine
import ETL.config as config

logger = logging.getLogger(__name__)

def create_database_connection() -> Engine:
    """
    Create and return a SQLAlchemy Engine for a SQL Server database connection.
    
    Returns:
        Engine: SQLAlchemy Engine connected to the SQL Server database
    """
    try:
        database_connection = URL.create(
            "mssql+pyodbc",
            username=config.DB_USER,
            password=config.DB_PASS,
            host=config.DB_SERVER,
            port=1433,  
            database=config.DB_NAME,
            query={
                "driver": "ODBC Driver 18 for SQL Server",
                "TrustServerCertificate": "yes",
                "authentication": "ActiveDirectoryIntegrated",
            },
        )
        engine = create_engine(database_connection)
        logger.info("Successfully created database connection engine.")
        return engine

    except Exception as e:
        logger.error(f"Failed to create database connection: {e}")
        raise 


In [ ]:
cards.head()

,card_id,card_brand,card_type,expires,has_chip,num_cards_issued,credit_limit,account_open_date,year_pin_last_changed,card_on_dark_web
0,4524,Visa,Debit,202212,YES,2,24295,200209,2008,No
1,2731,Visa,Debit,202012,YES,2,21968,201404,2014,No
2,3701,Visa,Debit,202402,YES,2,46414,200307,2004,No
3,42,Visa,Credit,202408,NO,1,12400,200301,2012,No
4,4659,Mastercard,Debit (Prepaid),200903,YES,1,28,200809,2009,No


In [ ]:
engine = create_database_connection()
cards.to_sql(
            name='dim_card',
            con=engine,
            schema='financial',
            if_exists='replace',  # Append to the table if it exists
            index=False,
            chunksize=1000  # Adjust chunk size as needed
        )

1130

In [ ]:
engine = create_database_connection()

dim_card = pd.read_sql('SELECT * FROM financial.dim_card', engine)

dim_card

,card_id,card_brand,card_type,expires,has_chip,num_cards_issued,credit_limit,account_open_date,year_pin_last_changed,card_on_dark_web
0,4524,Visa,Debit,202212,True,2,24295,200209,2008,True
1,2731,Visa,Debit,202012,True,2,21968,201404,2014,True
2,3701,Visa,Debit,202402,True,2,46414,200307,2004,True
3,42,Visa,Credit,202408,True,1,12400,200301,2012,True
4,4659,Mastercard,Debit (Prepaid),200903,True,1,28,200809,2009,True
5,4537,Visa,Credit,200309,True,1,27500,200309,2012,True
6,1278,Visa,Debit,202207,True,2,28508,201102,2011,True
7,3687,Mastercard,Debit,202206,True,2,9022,200307,2015,True
8,3465,Mastercard,Debit (Prepaid),202011,True,2,54,201006,2015,True
9,3754,Mastercard,Debit (Prepaid),202302,True,1,99,200607,2012,True


In [ ]:
import pandas as pd
from sqlalchemy.engine import Engine
import logging

logger = logging.getLogger(__name__)

def load_dataframe_to_sql(
    df: pd.DataFrame,
    table_name: str,
    engine: Engine,
    schema: str,
    if_exists: str = "append",
    index: bool = False
) -> None:
    """
    Load a Pandas DataFrame into a SQL Server table.

    Args:
        df (pd.DataFrame): The DataFrame to load.
        table_name (str): Name of the target SQL Server table.
        engine (Engine): SQLAlchemy Engine object.
        schema (str, optional): Target schema (e.g., 'dbo', 'financial').
        if_exists (str, optional): Behavior if table exists ('fail', 'replace', 'append').
        index (bool, optional): Whether to write DataFrame index as a column. Default is False.

    Raises:
        ValueError: If the DataFrame is empty.
        Exception: For other database-related errors.
    """
    
    if df.empty:
        logger.warning(f"DataFrame is empty. No data loaded into table '{table_name}'.")
        raise ValueError("DataFrame is empty. Aborting load.")

    try:
        df.to_sql(
            name=table_name,
            con=engine,
            schema=schema,
            if_exists='replace',  # Append to the table if it exists
            index=False,
            chunksize=1000  # Adjust chunk size as needed
        )    
        logger.info(f"Successfully loaded data into table '{schema + '.' if schema else ''}{table_name}'.")
    except Exception as e:
        logger.error(f"Error loading data into table '{table_name}': {e}")
        raise


In [ ]:
load_dataframe_to_sql(cards,'dim_card',engine,'financial')

In [ ]:
cards['has_chip'].str.lower().value_counts()

NameError: name 'cards' is not defined

In [ ]:
print(cards['card_id'].duplicated().sum())
print(cards.apply(lambda col: col.duplicated().sum()))


0
card_id                     0
card_brand               6142
card_type                6143
expires                  5887
has_chip                 6144
num_cards_issued         6143
credit_limit             2492
account_open_date        5843
year_pin_last_changed    6127
card_on_dark_web         6145
dtype: int64


In [ ]:
cards['card_on_dark_web'].value_counts(dropna=False,ascending=False)

card_on_dark_web
No    6146
Name: count, dtype: int64

In [ ]:
print(cards.dtypes)

card_id                   int64
card_brand               object
card_type                object
expires                  object
has_chip                 object
num_cards_issued          int64
credit_limit              int64
account_open_date        object
year_pin_last_changed     int64
card_on_dark_web         object
dtype: object


In [ ]:
cards.to_csv(r"C:\Users\jimmy\Documents\Data Projects\Project\Financial Transactions Project\data\cards.csv",index=False)

In [ ]:
cards['num_cards_issued'].value_counts(dropna=False)

num_cards_issued
1    3114
2    2972
3      60
Name: count, dtype: int64

In [ ]:
print(cards['num_cards_issued'].unique())
print(cards['num_cards_issued'].apply(type).value_counts())


[2 1 3]
num_cards_issued
<class 'int'>    6146
Name: count, dtype: int64


In [ ]:
cards

,card_id,card_brand,card_type,expires,has_chip,num_cards_issued,credit_limit,account_open_date,year_pin_last_changed,card_on_dark_web
0,4524,Visa,Debit,202212,YES,2,24295,200209,2008,No
1,2731,Visa,Debit,202012,YES,2,21968,201404,2014,No
2,3701,Visa,Debit,202402,YES,2,46414,200307,2004,No
3,42,Visa,Credit,202408,NO,1,12400,200301,2012,No
4,4659,Mastercard,Debit (Prepaid),200903,YES,1,28,200809,2009,No
...,...,...,...,...,...,...,...,...,...,...
6141,5361,Amex,Credit,202401,YES,1,6900,200011,2013,No
6142,2711,Visa,Credit,202101,YES,2,5700,201204,2012,No
6143,1305,Mastercard,Credit,202008,NO,2,9200,201202,2012,No
6144,743,Mastercard,Debit,202001,YES,1,28074,202001,2020,No


In [ ]:
import ETL.load_data as load
importlib.reload(load)
load.load_dataframe_to_sql(cards,'dim_card',engine,'financial')

In [ ]:
print(f'{cards.shape[0]} rows and {cards.shape[1]} columns')

6146 rows and 10 columns


In [ ]:
def extract_cards_data() -> pd.DataFrame:
    """
    This function extracts card data from a CSV file and returns it as a DataFrame.
    It logs the number of records loaded or any errors encountered during the process.
    The function uses the path defined in the config module to locate the CSV file.

    Args:
        None
    Returns:
        pd.DataFrame: DataFrame containing card data if successful, None otherwise.
    """
    try:
        df = pd.read_csv(config.CARDS_FILE)
        logger.info(f"✅ Extracted cards_data.csv with {len(df)} records")
        return df
    except Exception as e:
        logger.error(f"❌ Error extracting cards_data.csv: {e}")
        raise 



In [46]:
df = extract_cards_data()

In [37]:
df.sample(10)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
4427,1162,1117,Visa,Debit,4101718321165426,02/2020,388,YES,1,$9827,02/2008,2009,No
2482,4932,248,Mastercard,Debit,5499359671695760,11/2021,129,YES,1,$11749,10/1998,2008,No
2821,101,1813,Mastercard,Debit (Prepaid),5548793272738670,04/2024,233,YES,1,$41,01/2006,2010,No
473,2332,698,Visa,Debit,4401743622113558,12/2014,322,YES,1,$25214,03/2014,2014,No
4254,4104,1077,Visa,Debit (Prepaid),4849525193834905,05/2020,704,YES,2,$33,08/2001,2009,No
811,4080,223,Visa,Debit (Prepaid),4502772598432893,04/2024,928,YES,1,$55,07/2019,2019,No
1092,5786,1997,Amex,Credit,373196972571995,08/2022,333,YES,1,$25600,12/2003,2009,No
2460,1600,1539,Mastercard,Credit,5729767434317024,09/2020,422,YES,2,$6200,02/2020,2020,No
1003,4155,247,Mastercard,Debit,5711629583407537,01/2012,577,YES,2,$16751,08/2005,2010,No
5558,1456,1092,Discover,Credit,6731998965120910,07/2022,82,YES,2,$9600,02/2020,2020,No


In [40]:
import pandas as pd

# Sample data to test
data = {
    'card_id': [101, 102, 103, 104, 101, 106],
    'client_id': [1, 2, 3, 4, 1, 6],
    'credit_limit': [5000, 15000, 10000, 25000, 5000, 18000],
    'acct_open_date': ['2020-01', '2021-02', '2020-06', '2021-03', '2020-01', '2021-05'],
}

# Convert to DataFrame
df_cleaned = pd.DataFrame(data)

# Define the critical columns to check for duplicates
critical_columns = ['card_id', 'client_id']

# Count duplicates based on critical columns
duplicates_count = df_cleaned.duplicated(subset=critical_columns).sum()

# If duplicates are found, log the info and raise an error
if duplicates_count > 0:
    duplicates = df_cleaned.loc[df_cleaned.duplicated(subset=critical_columns), critical_columns]
    print(f"Found {duplicates_count} duplicate records based on columns {critical_columns}.")
    print(f"Duplicate records:\n{duplicates}")
    #raise ValueError(f'{duplicates_count} duplicate records found in CARDS data {critical_columns} columns.')




Found 1 duplicate records based on columns ['card_id', 'client_id'].
Duplicate records:
   card_id  client_id
4      101          1


In [14]:
import pandas as pd
def extract_data(file_path: str, data_type: str) -> pd.DataFrame:
    """
    Generic function to extract data from CSV files and return as DataFrame.
    Logs the number of records loaded or any errors encountered during the process.
    
    Args:
        file_path (str): Path to the CSV file to extract
        data_type (str): Name of the data type for logging purposes (e.g., 'cards', 'transactions', 'users')
    
    Returns:
        pd.DataFrame: DataFrame containing the extracted data
        
    Raises:
        Exception: Re-raises any exception encountered during CSV reading
    """
    try:
        df = pd.read_csv(file_path)
        logger.info(f"✅ Extracted {data_type}_data.csv with {len(df)} records")
        return df
    except Exception as e:
        logger.error(f"❌ Error extracting {data_type}_data.csv: {e}")
        raise

In [4]:
import logging
import ETL.extract_data as extract
import ETL.config as config
logger = logging.getLogger(__name__)

df = extract.extract_data(config.CARDS_FILE_LOCAL,'CARDS')
#df['expires'] = pd.to_datetime('01/' + df['expires'].astype(str), format='%d/%m/%Y', errors='raise')
#print(type(df['expires']))
df

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,5361,185,Amex,Credit,300609782832003,01/2024,663,YES,1,$6900,11/2000,2013,No
6142,2711,185,Visa,Credit,4718517475996018,01/2021,492,YES,2,$5700,04/2012,2012,No
6143,1305,1007,Mastercard,Credit,5929512204765914,08/2020,237,NO,2,$9200,02/2012,2012,No
6144,743,1110,Mastercard,Debit,5589768928167462,01/2020,630,YES,1,$28074,01/2020,2020,No


In [5]:
df['id'].sort_values().head()

2374    0
4265    1
3614    2
5466    3
291     4
Name: id, dtype: int64

In [30]:
df.loc[df['credit_limit'].str.contains(r'[$]', na=False)].head(10)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No
5,4537,1746,Visa,Credit,4404898874682993,09/2003,736,YES,1,$27500,09/2003,2012,No
6,1278,1746,Visa,Debit,4001482973848631,07/2022,972,YES,2,$28508,02/2011,2011,No
7,3687,1746,Mastercard,Debit,5627220683410948,06/2022,48,YES,2,$9022,07/2003,2015,No
8,3465,1746,Mastercard,Debit (Prepaid),5711382187309326,11/2020,722,YES,2,$54,06/2010,2015,No
9,3754,1746,Mastercard,Debit (Prepaid),5766121508358701,02/2023,908,YES,1,$99,07/2006,2012,No


In [26]:
df['credit_limit'].value_counts()
#check if there are floating point numbers in the credit_limit column
df['credit_limit'] = df['credit_limit'].astype(str).str.extract(r'(\d+)').astype(int)
df.loc[df['credit_limit'].between() , 'credit_limit']

Series([], Name: credit_limit, dtype: int64)